<a href="https://colab.research.google.com/github/Varunsunvatla/Chatbot/blob/main/ChatBot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import json
import tensorflow as tk
import numpy as np
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, GlobalAveragePooling1D
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder

In [3]:
with open('intents.json')as file:
  data = json.load(file)

In [4]:
training_sentences = []
classes = []
labels = []
responses = []
ignore_words = [ '?' , '!' ]

In [5]:
for intent in data['intents']:
    for pattern in intent['patterns']:
        training_sentences.append(pattern)
        classes.append(intent['tag'])
    responses.append(intent['responses'])

    if intent['tag'] not in labels:
        labels.append(intent['tag'])

num_classes = len(labels)

In [6]:
label_encoder = LabelEncoder()
label_encoder.fit(classes)
classes = label_encoder.transform(classes)

In [7]:
vocab_size = 1000
embedding_dim = 16
max_len = 20
oov_token = "<OOV>"

In [8]:
tokenizer = Tokenizer(num_words=vocab_size, oov_token=oov_token)
tokenizer.fit_on_texts(training_sentences)
word_index = tokenizer.texts_to_sequences(training_sentences)
sequences = tokenizer.texts_to_sequences(training_sentences)
padded_sequences= pad_sequences(sequences , truncating='post', maxlen= max_len )

In [9]:
model = tk.keras.Sequential(
    [
        tk.keras.layers.Embedding(vocab_size , embedding_dim , input_length = max_len),
        tk.keras.layers.GlobalAveragePooling1D(),
        tk.keras.layers.Dense(70, activation = 'relu'),
        tk.keras.layers.Dense(70, activation = 'relu'),
        tk.keras.layers.Dense( num_classes, activation ='sigmoid')
    ]
)
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 20, 16)            16000     
                                                                 
 global_average_pooling1d (  (None, 16)                0         
 GlobalAveragePooling1D)                                         
                                                                 
 dense (Dense)               (None, 70)                1190      
                                                                 
 dense_1 (Dense)             (None, 70)                4970      
                                                                 
 dense_2 (Dense)             (None, 38)                2698      
                                                                 
Total params: 24858 (97.10 KB)
Trainable params: 24858 (97.10 KB)
Non-trainable params: 0 (0.00 Byte)
____________________

In [10]:

model.compile(
    optimizer = 'adam',
    loss = 'sparse_categorical_crossentropy',
    metrics = ['accuracy']
)

In [11]:
model.fit(padded_sequences,np.array(classes), epochs = 500)

Epoch 1/500
13/13 [==============================] - 1s 3ms/step - loss: 3.6342 - accuracy: 0.0322 
Epoch 2/500
13/13 [==============================] - 0s 4ms/step - loss: 3.6236 - accuracy: 0.0718
Epoch 3/500
13/13 [==============================] - 0s 3ms/step - loss: 3.6101 - accuracy: 0.0668
Epoch 4/500
13/13 [==============================] - 0s 3ms/step - loss: 3.5869 - accuracy: 0.0668
Epoch 5/500
13/13 [==============================] - 0s 2ms/step - loss: 3.5550 - accuracy: 0.0668
Epoch 6/500
13/13 [==============================] - 0s 3ms/step - loss: 3.5210 - accuracy: 0.0668
Epoch 7/500
13/13 [==============================] - 0s 3ms/step - loss: 3.5015 - accuracy: 0.0668
Epoch 8/500
13/13 [==============================] - 0s 4ms/step - loss: 3.4912 - accuracy: 0.0668
Epoch 9/500
13/13 [==============================] - 0s 3ms/step - loss: 3.4814 - accuracy: 0.0668
Epoch 10/500
13/13 [==============================] - 0s 3ms/step - loss: 3.4736 - accuracy: 0.0668
Epoch 11

In [12]:
model.save("chat_bot")

In [13]:
import pickle

In [14]:
with open('tokenizer.pickle','wb') as handle:
  pickle.dump(tokenizer , handle , protocol = pickle.HIGHEST_PROTOCOL)

In [15]:
with open('lable_encoder.pickle', 'wb') as ecn_file:
  pickle.dump(label_encoder , ecn_file , protocol = pickle.HIGHEST_PROTOCOL)

In [16]:
pip install colorama

In [17]:
import json
import numpy as np
from tensorflow import keras
from sklearn.preprocessing import LabelEncoder
import colorama
colorama.init()
from colorama import Fore,Style,Back
import random
import pickle

In [18]:
with open('intents.json') as file:
  data = json.load(file)

def chatbot():
  model = keras.models.load_model('chat_bot')


with open('tokenizer.pickle','rb') as handle:
  tokenizer = pickle.load(handle)


with open('lable_encoder.pickle', 'rb') as ecn_file:
  lable_encoder = pickle.load(ecn_file)


max_len = 20

In [ ]:
while True:
  print(Fore.LIGHTBLUE_EX + "User: " + Style.RESET_ALL, end="")
  inp = input()
  if inp.lower() == "stop":
    break

  result = model.predict(keras.preprocessing.sequence.pad_sequences(tokenizer.texts_to_sequences([inp]), truncating = 'post', maxlen=max_len))
  tag = lable_encoder.inverse_transform([np.argmax(result)])

  for i in data['intents']:
    if i['tag'] == tag:
      print(Fore.GREEN + "Jarvis: " + Style.RESET_ALL , np.random.choice(i['responses']))

print(Fore.YELLOW + "start messaging with the bot (type stop to stop)" + Style.RESET_ALL)
chat()

User: hello
1/1 [==============================] - 0s 148ms/step
Jarvis:  At your service sir
User: what is my name
1/1 [==============================] - 0s 38ms/step
Jarvis:  I am a Chatbot.My name is Jarvis
User: thank you
1/1 [==============================] - 0s 42ms/step
Jarvis:  Our university offers Information Technology, computer Engineering, Mechanical engineering,Chemical engineering, Civil engineering and extc Engineering.
User: bye
1/1 [==============================] - 0s 23ms/step
Jarvis:  Talk to you later
User: 